## Import Modules

In [1]:
import torch, os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.autograd as autograd
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.utils import save_image
import numpy as np
from utils.visdom_utils import VisFunc

## Architectures
![infogan](http://nooverfit.com/wp/wp-content/uploads/2017/10/QQ%E6%88%AA%E5%9B%BE20171009174341.png)

## Front-end Feature Extractor in Discriminator  
<br>
InfoGAN의 Architecture에서 볼 수 있는 특이한 점은,  
Discriminator D와 Encoder Q가 서로 독립적인 네트워크가 아니라,  
네트워크의 일정 부분을 서로 공유하고 있다는 점입니다.  
<br>
D와 Q가 서로 공유하고 있는 부분을 보통 Front-End라고 부릅니다.  
<br>
true image와 fake image들은 우선 Front-End를 통과하게 되고,  
거기서 나온 feature들은 두 갈래로 나뉘어 D와 Q의 output으로 각각 나오게 됩니다. 
<br>
<br>
<br>
또한 DCGAN 이후로 다양한 GAN variants들에서 볼 수 있는 일반적인 테크닉들이 InfoGAN에도 적용되어 있습니다.  

* Discriminator에는 Leaky ReLU를, Generator에는 ReLU를 사용하는 것이 가장 일반적인 방식이 되었습니다.  
* Batch Normalization도 GAN에서 사용이 되고 있습니다. (그러나 최근에는 BN 말고도 더 다양한 Normalization을 사용하기도 합니다.)   
* Generator의 마지막 레이어와 Discriminator의 첫 번째 레이어는 BN을 적용하지 않습니다.

In [2]:
class FrontEnd(nn.Module):
    def __init__(self):
        super(FrontEnd, self).__init__()

        self.main = nn.Sequential(
            nn.Conv2d(1,64,4,2,1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(64,128,4,2,1,bias=False),
            nn.BatchNorm2d(128), 
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(128, 1024,7,bias=False),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
        )

    def forward(self,x):
        output = self.main(x)
        return output

## Discriminator D  
<br>
이미지의 True or False를 판별하기 위한 Feature들은 Front-End에서 충분히 뽑혔다고 판단한 것 같습니다.  
D의 네트워크는 매우 단순합니다

In [3]:
class Dmodel(nn.Module):
    def __init__(self):
        super(Dmodel, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1024,1,1),
            nn.Sigmoid()
        )

    def forward(self,x):
        output=self.main(x).view(-1,1)
        return output

## Classifier Q  
<br>
D와 마찬가지로 Q도 매우 단순한 구조로 되어 있습니다.  
Q는 discrete code와 continuous code를 예측해야하는데,  
discrete code는 CrossEntropy를 이용해서 reconstruction loss를 계산하기 때문에 logit을,  
continuous code는 Log Gaussian을 이용해서 reconstruction loss를 계산하기 때문에 mu와 var를 output하게 됩니다.

In [4]:
class Qmodel(nn.Module):
    def __init__(self):
        super(Qmodel,self).__init__()

        self.conv = nn.Conv2d(1024,128,1,bias=False) 
        self.bn = nn.BatchNorm2d(128)
        self.lReLU = nn.LeakyReLU(0.1, inplace=True)
        self.conv_disc = nn.Conv2d(128,10,1)
        self.conv_mu = nn.Conv2d(128,2,1)
        self.conv_var = nn.Conv2d(128,2,1)

    def forward(self,x):
        y = self.conv(x)
        disc_logits = self.conv_disc(y).squeeze()
        mu = self.conv_mu(y).squeeze()
        var = self.conv_var(y).squeeze().exp()
        return disc_logits, mu, var


## Generator G

In [5]:
class Gmodel(nn.Module):
    def __init__(self):
        super(Gmodel, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(74, 1024,1,1, bias=False), # noise 62 + discrete code 10 + continuous code 2 = 74
            nn.BatchNorm2d(1024),
            nn.ReLU(True),
            nn.ConvTranspose2d(1024, 128,7,1,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128,64,4,2,1,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64,1,4,2,1,bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.main(x)
        return output


## Initialize Models and Optimizer and Data Loader

In [6]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
    
# Models
FE=FrontEnd()
D=Dmodel()
Q=Qmodel()
G=Gmodel()

for i in [FE, D, Q, G]:
    i.cuda()
    i.apply(weights_init)

# Optimizers
optimD = optim.Adam([{'params':FE.parameters()},
                     {'params':D.parameters()}],
                    lr=0.0001, betas=(0.5, 0.99) )

optimG = optim.Adam([{'params':G.parameters()},
                     {'params':Q.parameters()}],
                    lr=0.0002, betas=(0.5, 0.99) )

# Datasets
batch_size = 100

# Train using 10K Test Images
train_data = dset.MNIST('./dataset', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)

# Test using 60K Train Images
test_data = dset.MNIST('./dataset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=False)

## Inputs(datasets, noises) and Visualization Tool

In [7]:
# fixed random variables for test
c0 = torch.linspace(-1,1,10).view(-1,1).repeat(10,0)
c1 = torch.stack((c0, torch.zeros(1).expand_as(c0)),1).cuda()
c2 = torch.stack((torch.zeros(1).expand_as(c0), c0),1).cuda()
one_hot = torch.eye(10).repeat(1,1,10).view(100,10).cuda()
fix_noise = torch.Tensor(100, 62).uniform_(-1, 1).cuda()

# random noises sampling function
def _noise_sample(dis_c, con_c, noise, bs):
    idx = np.random.randint(10, size=bs)
    c = np.zeros((bs, 10))
    c[range(bs),idx] = 1.0
    dis_c.data.copy_(torch.Tensor(c))
    con_c.data.uniform_(-1.0, 1.0)
    noise.data.uniform_(-1.0, 1.0)
    z = torch.cat([noise, dis_c, con_c], 1).view(-1, 74, 1, 1)
    return z, idx

# Visdom
env_name = 'infoGAN'
vf = VisFunc(enval=env_name)

# Generated Image Folder
if not os.path.exists('tmp') : os.makedirs('tmp')

## Define Losses

![log_gaussian](https://user-images.githubusercontent.com/613623/30778123-7328abc0-a0cd-11e7-8998-7e4ef07cc25f.png)
https://user-images.githubusercontent.com/613623/30778123-7328abc0-a0cd-11e7-8998-7e4ef07cc25f.png

In [8]:
# Define Losses
class log_gaussian:
    def __call__(self, x, mu, var):
        logli = -0.5*(var.mul(2*np.pi)+1e-6).log() - (x-mu).pow(2).div(var.mul(2.0)+1e-6)
        return logli.sum(1).mean().mul(-1)

criterionD = nn.BCELoss()
criterionQ_dis = nn.CrossEntropyLoss()
criterionQ_con = log_gaussian()

## Functions for Classification Test

In [ ]:
def set_mode(mode='train'):
    if mode == 'train' : 
        G.train()
        FE.train()
        D.train()
        Q.train()
    elif mode == 'eval' :
        G.eval()
        FE.eval()
        D.eval()
        Q.eval()
    else : raise BaseException('wrong mode') 
    
def test(num_class=10):
        '''
        1. 클러스터링 결과를 2D 테이블로 만든다
        2. 테이블을 이용해서 클러스터가 얼마나 잘 뭉쳤는지 계산한다(hungarian algorithm 이용)
        
        Table Example : 
        
             true
             label 0     1     2     3     4     5     6     7     8     9
     cluster  | - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        0     |    0     1     0     1     0     3     5     1     0   969
        1     |    0     3     1     2     7   613     4     0   505     0
        2     |    5   547     3     3   461     5     1     0     5     2
        3     |    8     5     0   950    17     1     0    25     2     2
        4     |    2   184   774     0     0     6     6     0    10     0
        5     |    1     1     0     3     3     1     5   872     4     2
        6     |    0     4     2     0     0     4   927    13     0     8
        7     |  684    13    88     1    11     2     0     2   226     1
        8     |    2    13     3     7   592   317     7    15    12     6
        9     |  302   158   273    14     5     5     1     7   241     3
        
        위 테이블을 보면, 클러스터 0은 9라는 숫자가 제일 많이 차지하고 있는 것을 알 수 있다.( 첫 번째 행 )
        또한 클러스터 3은 3이라는 숫자가 제일 많이 차지하고 있는 것을 알 수 있다. ( 네 번째 행 )
        
        클러스터 0, 3, 5, 6 처럼 클러스터링이 잘 된 곳이 있는 반면,
        클러스터 1, 2, 4, 7, 8, 9는 여러 종류의 숫자가 섞여 있는 것을 알 수 있다.
        
        이러한 테이블을 만들어 놓고 hungarian algorithm을 이용하면 clustering performance를 계산할 수 있다.
        '''
        
        import torch.nn.functional as F
        from scipy.optimize import linear_sum_assignment
        
        set_mode('eval')
        
        LABEL = [] # 각 트레이닝 샘플의 label을 저장
        PREDICT = [] # 각 트레이닝 샘플의 encoder prediction 결과를 저장
        TABLE = [[0 for i in range(num_class)] for k in range(num_class)] # LABEL과 PREDICT를 종합해서 위에서 처럼 2D 테이블을 만들어야 한다.
        total = 0
        for batch_idx, (images,labels) in enumerate(test_loader):
            real_x = Variable(images.cuda()) # test image 뽑아서
            fe_out = FE(real_x) # FE에 넣고
            q_logits, _, _ = Q(fe_out) # discrete code의 logit을 뽑은 뒤
            q_softmax = F.softmax(q_logits) # softmax를 통과시켜
            q_index = q_softmax.max(1)[1] # 가장 likely한 클러스터를 찾는다.
            
            PREDICT.append(q_index.data) # 각 트레이닝 샘플의 클러스터와 실제 레이블을 차곡차곡 쌓는다.
            LABEL.append(labels)
            total += labels.size(0)

        # 위에서 쌓은 결과를 이용해서 테이블을 채운다.
        LABEL = torch.cat(LABEL,0)
        PREDICT = torch.cat(PREDICT,0).cpu()
        for idx, label in enumerate(LABEL):
            TABLE[label][PREDICT[idx]] += 1

        # 테이블을 이용해서 hungarian algorithm을 수행한다.
        TABLE = torch.FloatTensor(TABLE)
        row, col = linear_sum_assignment(-TABLE.numpy())
        acc = TABLE.numpy()[row,col].sum()/total

        print(TABLE)
        print('[TEST RESULT] : ACC : {:.4f}%'.format(acc*100))

        set_mode('train')

## Training

In [ ]:
for epoch in range(100):
    if (epoch % 5) == 0 :  test(num_class=10)
    for num_iters, batch_data in enumerate(train_loader,0):

        # real part
        optimD.zero_grad() # D를 학습시켜야 하므로 우선 gradient를 초기화 시켜주고

        x, _ = batch_data # 이미지를 불러온 뒤
        real_x = Variable(x.cuda()) # Variable로 만들어 준다.
        label = Variable(torch.ones(batch_size).float().cuda(), requires_grad=False) # real 이미지에 대해서는 BCE Loss 정답을 1로 만들어 준다.

        fe_out1 = FE(real_x) # 이미지를 FE에 넣어서 feature를 뽑고
        probs_real = D(fe_out1) # 이 feature를 다시 D에 넣어서 true/false 값을 얻는다.
        label.data.fill_(1) # real 이미지에 대해서는 BCE Loss 정답을 1로 만들어 준다.
        loss_real = criterionD(probs_real, label) # Loss 계산한 뒤
        loss_real.backward() # D를 학습시킨다.

        # fake part
        # fake 이미지에 대해서도 D를 트레이닝 시켜준다.
        dis_c = Variable(torch.FloatTensor(batch_size,10).cuda()) # 랜덤하게 discrete code와
        con_c = Variable(torch.FloatTensor(batch_size,2).cuda()) # continuous code,
        noise = Variable(torch.FloatTensor(batch_size,62).cuda()) # 그리고 noise vector를 먼저 만든 뒤에
        z, idx = _noise_sample(dis_c,con_c,noise,batch_size) # 각각 categorical distribution, uniform distribution, uniform distribution에서 초기화해준다.

        fake_x = G(z) # z는 dis_c, con_c, noise가 합쳐진 값이다.
        fe_out2 = FE(fake_x.detach()) # real 이미지와 동일한 과정을 거친다.
        probs_fake = D(fe_out2)
        label.data.fill_(0)
        loss_fake = criterionD(probs_fake, label)
        loss_fake.backward()

        D_loss = loss_real + loss_fake
        optimD.step()

        # G and Q part
        # G 학습은 fake 이미지에 대해서 D와 반대로 학습시켜주면 된다.
        optimG.zero_grad()

        fe_out = FE(fake_x)
        probs_fake = D(fe_out)
        label.data.fill_(1.0)
        reconstruct_loss = criterionD(probs_fake, label)

        # InfoGAN의 Mutual Information Maximization을 optimization 하는 부분
        q_logits, q_mu, q_var = Q(fe_out)
        class_ = torch.LongTensor(idx).cuda()
        target = Variable(class_)
        dis_loss = criterionQ_dis(q_logits, target) # discrete code에 대해서는 Cross Entropy를 계산하고
        con_loss = criterionQ_con(con_c, q_mu, q_var)*0.1 # continuous code에 대해서는 Log Gaussian을 계산해준다.

        G_loss = reconstruct_loss + dis_loss + con_loss
        G_loss.backward()
        optimG.step()

        if num_iters % 50 == 0:
            print('Epoch:{}, Iter:{}, Dloss: {:.3f}, Gloss: {:.3f}, Preal: {:.3f}, Pfake: {:.3f}'.format(
                epoch, num_iters, D_loss.data[0],
                G_loss.data[0], probs_real.data.mean(), probs_fake.data.mean())
            )

            z = Variable(torch.cat([fix_noise, one_hot, c1], 1).view(-1, 74, 1, 1))
            x_save = G(z)
            title1 = '(C1)'+str(epoch)+'_'+str(num_iters)
            save_image(x_save.data, 'tmp/'+title1+'.png', nrow=10)
            vf.imshow_multi(x_save.data.cpu(), nrow=10, title=title1,factor=1)

            z = Variable(torch.cat([fix_noise, one_hot, c2], 1).view(-1, 74, 1, 1))
            x_save = G(z)
            title2 = '(C2)'+str(epoch)+'_'+str(num_iters)
            save_image(x_save.data, 'tmp/'+title2+'.png', nrow=10)
            vf.imshow_multi(x_save.data.cpu(), nrow=10, title=title2,factor=1)


    0     0   957     0     0     0    23     0     0     0
    0     0  1104     0     0     0    31     0     0     0
    0     0   934     0     0     0    98     0     0     0
    0     0   888     0     0     0   122     0     0     0
    0     0   526     0     0     0   456     0     0     0
    0     0   706     0     0     0   186     0     0     0
    0     0   730     0     0     0   228     0     0     0
    0     0  1008     0     0     0    20     0     0     0
    0     0   750     0     0     0   224     0     0     0
    0     0   731     0     0     0   278     0     0     0
[torch.FloatTensor of size 10x10]

[TEST RESULT] : ACC : 15.6000%


/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch:0, Iter:0, Dloss: 1.431, Gloss: 3.204, Preal: 0.549, Pfake: 0.522
Epoch:0, Iter:50, Dloss: 1.145, Gloss: 1.060, Preal: 0.571, Pfake: 0.421
Epoch:0, Iter:100, Dloss: 1.002, Gloss: 0.982, Preal: 0.612, Pfake: 0.375
Epoch:0, Iter:150, Dloss: 0.882, Gloss: 1.062, Preal: 0.638, Pfake: 0.321
Epoch:0, Iter:200, Dloss: 0.698, Gloss: 1.296, Preal: 0.712, Pfake: 0.273
Epoch:0, Iter:250, Dloss: 0.599, Gloss: 1.491, Preal: 0.743, Pfake: 0.236
Epoch:0, Iter:300, Dloss: 0.579, Gloss: 1.431, Preal: 0.738, Pfake: 0.216
Epoch:0, Iter:350, Dloss: 0.476, Gloss: 1.591, Preal: 0.787, Pfake: 0.186
Epoch:0, Iter:400, Dloss: 0.396, Gloss: 1.677, Preal: 0.833, Pfake: 0.168
Epoch:0, Iter:450, Dloss: 0.368, Gloss: 1.829, Preal: 0.831, Pfake: 0.148
Epoch:0, Iter:500, Dloss: 0.350, Gloss: 1.909, Preal: 0.845, Pfake: 0.145
Epoch:0, Iter:550, Dloss: 0.333, Gloss: 1.986, Preal: 0.851, Pfake: 0.136
Epoch:1, Iter:0, Dloss: 0.253, Gloss: 1.976, Preal: 0.911, Pfake: 0.129
Epoch:1, Iter:50, Dloss: 0.332, Gloss: 2.02

Epoch:8, Iter:350, Dloss: 1.043, Gloss: 0.967, Preal: 0.623, Pfake: 0.369
Epoch:8, Iter:400, Dloss: 1.106, Gloss: 0.971, Preal: 0.595, Pfake: 0.369
Epoch:8, Iter:450, Dloss: 1.133, Gloss: 1.021, Preal: 0.606, Pfake: 0.372
Epoch:8, Iter:500, Dloss: 1.115, Gloss: 1.014, Preal: 0.574, Pfake: 0.354
Epoch:8, Iter:550, Dloss: 1.008, Gloss: 0.962, Preal: 0.657, Pfake: 0.365
Epoch:9, Iter:0, Dloss: 1.050, Gloss: 0.991, Preal: 0.625, Pfake: 0.361
Epoch:9, Iter:50, Dloss: 1.089, Gloss: 1.001, Preal: 0.595, Pfake: 0.362
Epoch:9, Iter:100, Dloss: 1.096, Gloss: 0.986, Preal: 0.584, Pfake: 0.360
Epoch:9, Iter:150, Dloss: 0.998, Gloss: 0.991, Preal: 0.628, Pfake: 0.363
Epoch:9, Iter:200, Dloss: 1.048, Gloss: 0.944, Preal: 0.646, Pfake: 0.388
Epoch:9, Iter:250, Dloss: 1.037, Gloss: 1.124, Preal: 0.619, Pfake: 0.351
Epoch:9, Iter:300, Dloss: 1.055, Gloss: 1.113, Preal: 0.597, Pfake: 0.331
Epoch:9, Iter:350, Dloss: 1.096, Gloss: 1.015, Preal: 0.594, Pfake: 0.360
Epoch:9, Iter:400, Dloss: 1.133, Gloss: 0

Epoch:16, Iter:150, Dloss: 0.935, Gloss: 1.151, Preal: 0.618, Pfake: 0.319
Epoch:16, Iter:200, Dloss: 0.952, Gloss: 1.066, Preal: 0.624, Pfake: 0.337
Epoch:16, Iter:250, Dloss: 1.103, Gloss: 1.101, Preal: 0.552, Pfake: 0.335
Epoch:16, Iter:300, Dloss: 1.003, Gloss: 1.135, Preal: 0.648, Pfake: 0.325
Epoch:16, Iter:350, Dloss: 0.977, Gloss: 1.260, Preal: 0.703, Pfake: 0.301
Epoch:16, Iter:400, Dloss: 0.812, Gloss: 1.278, Preal: 0.688, Pfake: 0.277
Epoch:16, Iter:450, Dloss: 1.163, Gloss: 1.095, Preal: 0.596, Pfake: 0.326
Epoch:16, Iter:500, Dloss: 1.033, Gloss: 1.087, Preal: 0.616, Pfake: 0.330
Epoch:16, Iter:550, Dloss: 0.968, Gloss: 1.070, Preal: 0.682, Pfake: 0.348
Epoch:17, Iter:0, Dloss: 0.969, Gloss: 1.281, Preal: 0.617, Pfake: 0.293
Epoch:17, Iter:50, Dloss: 0.985, Gloss: 1.183, Preal: 0.623, Pfake: 0.313
Epoch:17, Iter:100, Dloss: 0.927, Gloss: 1.161, Preal: 0.590, Pfake: 0.313
Epoch:17, Iter:150, Dloss: 0.900, Gloss: 1.165, Preal: 0.671, Pfake: 0.313
Epoch:17, Iter:200, Dloss: 0

Epoch:24, Iter:400, Dloss: 0.855, Gloss: 1.604, Preal: 0.701, Pfake: 0.220
Epoch:24, Iter:450, Dloss: 0.803, Gloss: 1.470, Preal: 0.639, Pfake: 0.247
Epoch:24, Iter:500, Dloss: 0.820, Gloss: 1.532, Preal: 0.751, Pfake: 0.227
Epoch:24, Iter:550, Dloss: 0.861, Gloss: 1.501, Preal: 0.660, Pfake: 0.257

    1     2     0     0     1     1   956     0    16     3
   16     1  1107     0     0     9     0     0     0     2
    8    10     0     0     0   998     3     3     4     6
  547     6     0     2   404    15     1     0    23    12
   10     3     0    18     0     4     0   941     0     6
  138     1     0     5   570     1     2     1   170     4
  223     2     1     0     2     6     2     1   721     0
    2     1     8     8     2    32     1     2     3   969
   34   885     0     9     9     5     1     4    25     2
    5     3     2   923     9     1    10    28    12    16
[torch.FloatTensor of size 10x10]

[TEST RESULT] : ACC : 86.1700%
Epoch:25, Iter:0, Dloss: 0.772, G